### model brand

In [ ]:
import pandas as pd
import re

In [ ]:
def load_df_products(name):
    return pd.read_csv(f'../data/processed/data_beer_{name}.csv')

In [4]:
data_beer = load_df_products('total')

In [5]:
data_beer.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket,id
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,heineken lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour,0
1,14.80,1.60,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,cruzcampo pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour,1
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,el aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour,2
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,carrefour holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour,3
4,14.15,1.53,Cerveza Mahou Clásica pack de 28 latas de 33 cl.,No promotion,mahou clásica,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour,4


In [6]:
def change_brand(brand_raw):
    names_brands = '''ramblers|amstel|mahou|dia|heineken|san miguel|estrella galicia|voll-damm|1906|budweiser|
    |cruzcampo|buckler|dia shandy|desperado|grimbergen|franziskaner|paulaner|corona|guinness|el aguila|
    |ambar|alhambra|coronita|leffe|kronenbourg|carlsberg|la salve|warsteiner|mexicana|pacífico|guiness|
    |founders|spatem|beck|cubanisto|aurum|keler|woll-damm|aurum|ambar|free damm|ladrón de manzanas|
    |brewdog|ipa lagunitas|inedit|daura|oro|estrella damm|damm|complot|corgon gard|judas|olañeta|
    |grevensteiner|inedit|chimay|house 13|lorea ipa boga|trappe|mort subite|kirin|urquell|cruz campo|
    |malquerida|moretti|clausthaler|affligem|stella artoi|sol|quilmes|daura marzen|turia|boga|
    |schöfferhofer|grolsch|la goudale|atkien dunkel|chérie|belzebuth|la virgen|lowenbrau|bulmers|jai alai|
    |strongbow|cruz del sur|voll damm|estrella levante|xibeca|victoria|estrella del sur|aliada|moritz|
    |brabante|superbock|kirin ichiban|staropramen|waterloo|weltenburger|schwaben|dab|praga|abbot|
    |lowenbräu|spaten|hofbräu|dos equis|birra moretti|karmeliet|kwak|gordon|martin's|delirium tremens|veltins|
    |pacifico|triple secret des moine|la bière du demon|baltika|molino viejo|cusqueña|cobra|zywiec|barista|
    |bischofshof|hoegaarden|schofferhofer|erdinger|konig ludwig|casimiro mahou|rosita|burro de sancho|
    |monkey|arriaca|ladron de manzanas|kopparberg|la sagra|sierra nevada|montseny|maisel & friends|
    |tyris|rabiosa|jaira|forastera|cibeles|mulhacen|gredos|ballut|madri chulapo|gastheiz|quijota|
    |marijuana|moli balear|dolina|icue|bizantina|bavaria|newcastle|timmermans|modelo|heifer|holbrand|
    |rubai|el águila|estrella de galicia|chouffe|peroni|miller|goya|super bock|sam miguel|karamalz|
    |skol|budejovicky|wersteiner|duvel|schneider weisse|john smith's|blue moon|rochefort-8|salitos|
    |old empire|köning ludwing weissbier|könig ludwig weissbier|blanche de charleroi|ruddles|
    |madrí chulapo|abbaye d'aulne|g de goudale|cuzqueña|london pride|wychwood hobgoblin|sureña|
    |innis&gunn|intense gold|faxe|bitburger|lindemans|maisel&friends|pearl jet|the one|original intense|
    |licher|tennent's|tedeum|enigma|mulhacén|gervensteiner|abbaye du lys|greventeiner|liefmans|rubia 4|
    |san sebastian|maisel|carrefour|koenigsbier|polar|coors|kadabra|brooklyn|alsfelder|madriz|presidente|
    |fentiman's de jengibre|lammsbräu|riedenburger|benediktinerabtei|iron maiden|old speckled hen|
    |ordio minero|rubia dab|corte ingles|thatchers gold'''
  
    brand = re.findall(names_brands,brand_raw)
    return brand[0]

In [7]:
data_beer['brand'].apply(change_brand)

0        heineken
1       cruzcampo
2       el aguila
3       carrefour
4           mahou
          ...    
1561    cruzcampo
1562    la virgen
1563        ambar
1564    lowenbrau
1565     brooklyn
Name: brand, Length: 1566, dtype: object

In [8]:
data_beer['brand'] = data_beer['brand'].apply(change_brand)

In [9]:
def duplicate_brand_error(data):
    if data == 'guiness':
        return 'guinness'
    elif data == 'cruz campo':
        return 'cruzcampo'
    elif data == 'voll damm':
        return 'voll-damm'
    elif data == 'schofferhofer':
        return 'schöfferhofer'
    elif data == 'ladron de manzanas':
        return 'ladrón de manzanas'
    elif data == 'el aguila':
        return 'el águila'
    elif data == 'estrella de galicia':
        return 'estrella galicia'
    elif data == 'superbock':
        return 'super bock'
    elif data == 'sam miguel':
        return 'san miguel'
    elif data == 'madri chulapo':
        return 'madrí chulapo'
    elif data == ('maisel' or 'maisel&friends'):
        return 'maisel & friends'
    elif data == 'köning ludwing weissbier':
        return 'könig ludwig weissbier'
    else:
        return data

In [10]:
data_beer['brand'] = data_beer['brand'].apply(duplicate_brand_error)

In [11]:
data_beer['brand'].value_counts()

mahou           151
san miguel      120
amstel           87
cruzcampo        80
ambar            71
               ... 
coors             1
riedenburger      1
polar             1
rubai             1
salitos           1
Name: brand, Length: 212, dtype: int64

In [ ]:
f'../beer_images/{url[0]}'
/home/ivan/Documentos/IRONHACK/Bootcamp/final_project/Birras_Pipol_House/beer_images/


In [13]:
data_beer_images= pd.DataFrame(columns=['id','local','brand'])

In [14]:
data_beer_images['id'] = data_beer['id']

In [22]:
data_beer_images['brand'] = data_beer['brand']

In [17]:
def local_beer(local):
    return f'/home/ivan/Documentos/IRONHACK/Bootcamp/final_project/Birras_Pipol_House/beer_images/{local}'

In [18]:
data_beer_images['local'] = data_beer_images['id'].apply(local_beer)

In [23]:
data_beer_images.head()

,id,local,brand
0,0,/home/ivan/Documentos/IRONHACK/Bootcamp/final_...,heineken
1,1,/home/ivan/Documentos/IRONHACK/Bootcamp/final_...,cruzcampo
2,2,/home/ivan/Documentos/IRONHACK/Bootcamp/final_...,el águila
3,3,/home/ivan/Documentos/IRONHACK/Bootcamp/final_...,carrefour
4,4,/home/ivan/Documentos/IRONHACK/Bootcamp/final_...,mahou


In [29]:
def increment_string(strng):
    number = re.findall('[0-9]+',strng)
    word = re.findall('[a-zA-Z]+',strng)
    if len(number)>0:
        new_number = int(number[0]) + 1
    else:
        new_number = 1
    
    print(len(str(new_number)))
    print(len(number))
          
    if len(str(new_number))!=len(number[0]):
        zeros = len(number[0])-len(str(new_number))
        new_number = '0'*zeros + str(new_number)
            
    new_string = word[0] + str(new_number)
    return new_string

In [30]:
increment_string('')

1
0


IndexError: list index out of range

In [16]:
re.findall('[0-9]+','foo11')

['11']

In [ ]:
import re
def increment_string(strng):
    number = re.findall('[0-9]+',strng)
    word = re.findall('[^0-9]+',strng)

    if len(number)>0:
        new_number = int(number[-1]) + 1
    else:
        new_number = 1
        
    if len(number)!=0:
        if len(str(new_number))!=len(number[-1]):
            zeros = len(number[-1])-len(str(new_number))
            new_number = '0'*zeros + str(new_number)
    if len(word)!=0:
        new_string = word[-1] + str(new_number)
    else:
        new_string = str(new_number)
    return new_string
